# MADDPG - Tennis

---

In this notebook, we will train an agent to navigate the Unity ML-Agents Tennis environment. In this environment, two agents control rackets to bounce a ball over a net. The goal of each agent is to keep the ball in play. The training algorithm that is used for training is MADDPG, implemented in PyTorch.

### 1. Import Required Packages

We begin by importing some necessary packages. If the code cell below returns an error, please revisit the project instructions on the README.md file.

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
from maddpg import MADDPG
import torch

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

We set the hyperparameters we will use later during the MADDPG implementation. The values are chosen using trial and error and from RL baselines

In [ ]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 256        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-2              # for soft update of target parameters
LR = 2e-4               # learning rate of the actor and of the critic
WEIGHT_DECAY = 0.0      # L2 weight decay

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [ ]:
env = UnityEnvironment(file_name='Tennis.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Train Agent

The code below initiates the agents and starts the training loop. Once the average reward over 100 episodes reaches 0.5, the training stops and for each agent(i), the weights are saved in  **_checkpoint\_maddpg\_actor(i).pth_** for the actor network and **_checkpoint\_maddpg\_critic(i).pth_** for the critic network.

In [ ]:
# initialise agents
agents = MADDPG(
    state_size=state_size, 
    action_size=action_size, 
    num_agents=num_agents, 
    random_seed=0,
    buffer_size = BUFFER_SIZE,
    batch_size = BATCH_SIZE,
    gamma = GAMMA,
    tau = TAU,
    lr = LR,
    weight_decay = WEIGHT_DECAY
)

In [ ]:
def maddpg(n_episodes=1000):
   
    total_scores = []                                       # list containing scores from each episode
    scores_window = deque(maxlen=100)                       # last 100 scores
    agents.reset()                                          # reset both agents noise
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]   # reset the environment
        states = env_info.vector_observations               # get the current state (for each agent)          
        scores = np.zeros(num_agents)                       # initialize the score (for each agent)
        while True:
            actions = agents.act(states)                    # select an action (for each agent)
            env_info = env.step(actions)[brain_name]        # send all actions to tne environment
            next_states = env_info.vector_observations      # get next state (for each agent)  
            rewards = env_info.rewards                      # get reward (for each agent)  
            dones = env_info.local_done                     # see if episodes finished
            agents.step(states, actions, rewards, next_states, dones)
            scores += rewards
            states = next_states                                          
            if np.any(dones):
                break 
        
        score = np.max(scores)         # get the max reward from the two agent rewards
        scores_window.append(score)    # save most recent score
        total_scores.append(score)     # save scores without deque to plot later

        print('\rEpisode {}\tAverage Score: {:.4f}'.format(i_episode, np.mean(scores_window), end=""))

        if np.mean(scores_window) >= 0.5:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.4f}'.format(i_episode, np.mean(scores_window)))
            for i, Agent in enumerate(agents.agents):
                torch.save(Agent.actor_local.state_dict(),  f"checkpoint_maddpg_actor_agent_{i}.pth")
                torch.save(Agent.critic_local.state_dict(), f"checkpoint_maddpg_critic_agent_{i}.pth")
            break
        
    return total_scores

scores = maddpg()    

### 4. Plot Scores

Once training has finished we can plot the average reward across all episodes.

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 5. Test Trained Agent on Environment
Once this cell is executed, we can watch the agents' performance for one episode.

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]            # reset the environment    
states = env_info.vector_observations                         # get the current state (for each agent)
scores = np.zeros(num_agents)                                 # initialize the score (for each agent)
while True:
    actions = agents.act(states)                               # select an action (for each agent)
    env_info = env.step(actions)[brain_name]                   # send all actions to tne environment
    next_states = env_info.vector_observations                 # get next state (for each agent)
    rewards = env_info.rewards                                 # get reward (for each agent)
    dones = env_info.local_done                                # see if episode finished
    scores += env_info.rewards                                 # update the score (for each agent)
    states = next_states                                       # roll over states to next time step
    if np.any(dones):                                          # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

### 6. Load Saved Weights and Test on Environment
Load the checkpoint weights to check the saved agents' performance for one episode.

In [ ]:

# load the weights from file
for i, Agent in enumerate(agents.agents):
    Agent.actor_local.load_state_dict(torch.load(f"checkpoint_maddpg_actor_agent_{i}.pth"))
    Agent.critic_local.load_state_dict(torch.load(f"checkpoint_maddpg_critic_agent_{i}.pth"))

env_info = env.reset(train_mode=False)[brain_name]            # reset the environment    
states = env_info.vector_observations                         # get the current state (for each agent)
scores = np.zeros(num_agents)                                 # initialize the score (for each agent)
while True:
    actions = agents.act(states)                               # select an action (for each agent)
    env_info = env.step(actions)[brain_name]                   # send all actions to tne environment
    next_states = env_info.vector_observations                 # get next state (for each agent)
    rewards = env_info.rewards                                 # get reward (for each agent)
    dones = env_info.local_done                                # see if episode finished
    scores += env_info.rewards                                 # update the score (for each agent)
    states = next_states                                       # roll over states to next time step
    if np.any(dones):                                          # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))  

When finished, you can close the environment.

In [ ]:
env.close()